# Geração do dataset
O dataset gerado por este scrip está separado em treino/teste e organizado por classes, pronto para ser utilizado como generator do Tensorflow.
Foi criado uma classe Dataset, de onde cada dataset herda uma classe específica. Ex.: o dataset "covid-chestxray-dataset (<https://github.com/ieee8023/covid-chestxray-dataset>)" é instanciado pela classe *cohen()*.

O pipeline de processamento é:<br>
1 - import das bibliotecas<br>
2 - instanciação dos objetos de cada dataset<br>
3 - Rotina de leitura de cada dataset, aplicando as devidas funções<br>
Obs.: nem todos os datasets possuem funções de prefiltragem/posfiltragem. Essas funções foram definidas conforme o processamento feito em <https://github.com/lindawangg/COVID-Net><br>
4 - Junção das tabelas de cada ds em uma só (com excessão do RSNA)<br>
5.1a - Filtragem das classes de interesse na tabela de imagens<br>
5.1b - Separação de imagens específicas para teste, conforme <https://github.com/lindawangg/COVID-Net><br>
5.2a - Filtragem das classes de interesse na tabela do RSNA<br>
5.2b - Aplica *split* no dataset RSNA<br>
6 - Junta as tabelas de treino e teste<br>
7 - Monta o dataset no path destino, copiando as imagens que já estão em formato de leitura e escrevendo as imagens em ```.dcm```<br>
<br>
Obs.: as estapas 3 e 7 podem demorar consideravelmente devido ao dataset RSNA que possui mais de 15k imagens. A fim de verificar a validade do script recomenda-se rodá-lo sem este dataset.<br>
<br>
As tables ou tabelas referidas nesse script são listas de dicionário do tipo \[{"path": target_path, "filename": filename, "class": finding, "url": url, "id": patientid}\], onde:<br>
- target_path = localização da imagem dentro do diretorio do dataset;<br>
- filename = nome do arquivo, presente no dataset;<br>
- class = classe de classificação da imagem. Ex.: Normal, COVID-19, etc;<br>
- url = URL da imagem, usado para detecção de imagens presentes simultaneamente em dois ou mais datasets. Se não há "url" no dataset, o valor None é preenchido na tabela;<br>
- patientid = utilizado para busca rapida de alguns paciente. Se não há "patientid" no dataset, o nome do arquivo sem extensão é utilizado.<br>



## IMPORT DA LIBS E FUNÇÕES

In [1]:
import numpy as np
import pandas as pd
import os
import random 
import pydicom as dicom
import cv2

In [2]:
from datasets import rsna, actualmed, cohen, fig1, sirm
from ds_utils import filter_table, split_table, table_info, remove_dupl_field

## CONSTROI OS OBJ DE CADA DS

In [3]:
datasets_path = os.path.join(os.getcwd(), "datasets")

cohen = cohen(datasets_path)
rsna = rsna(datasets_path)
actualmed = actualmed(datasets_path)
fig1 = fig1(datasets_path)
sirm = sirm(datasets_path)

## LE CADA DS E CONSTROI SUA TABELA APLICANDO AS FUNCOES NECESSARIAS

In [4]:
datasets = [cohen, actualmed, fig1, sirm, rsna]
for ds in datasets:
    ds.read()
    ds.prefilter()
    ds.mount_table()
    ds.postfilter()
    ds.mount_count_table()
    if ds.__name__ is sirm.__name__:
        remove_dupl_field(sirm, cohen, "url")

## JUNTA AS TABELAS EM UMA SÓ

In [5]:
target_ds = [cohen, actualmed, fig1, sirm]
file_table = []
for ds in target_ds:
    print(f"Dataset: {ds.__name__.upper()}")
    print("-----------------------------------------")
    print(f"Imagens: {ds.count}")
    print(f"Contagem de cada classe por dataset: {ds.count_table}\n")
    file_table += ds.table

print(f"Total de imagens: {len(file_table)}")

Dataset: COHEN
-----------------------------------------
Imagens: 763
Contagem de cada classe por dataset: {'COVID-19': 456, 'ARDS': 5, 'SARS': 16, 'Pneumocystis': 24, 'Streptococcus': 18, 'No Finding': 17, 'Chlamydophila': 1, 'E.Coli': 4, 'COVID-19, ARDS': 12, 'Klebsiella': 9, 'Legionella': 7, 'Unknown': 1, 'Pneumonia': 17, 'Lipoid': 8, 'Varicella': 5, 'Bacterial': 2, 'Mycoplasma Bacterial Pneumonia': 5, 'Influenza': 3, 'Cryptogenic Organizing Pneumonia': 10, 'Lobar Pneumonia': 5, 'Multilobar Pneumonia': 3, 'Organizing Pneumonia': 1, 'Eosinophilic Pneumonia': 2, 'Unusual Interstitial Pneumonia': 1, 'Lymphocytic Interstitial Pneumonia': 2, 'Desquamative Interstitial Pneumonia': 1, 'todo': 83, 'Spinal Tuberculosis': 1, 'Swine-Origin Influenza A (H1N1) Viral Pneumonia': 1, 'Tuberculosis': 10, 'Invasive Aspergillosis': 1, 'Herpes pneumonia': 2, 'Herpes pneumonia, ARDS': 1, 'Accelerated Phase Usual Interstitial Pneumonia': 2, 'Round pneumonia': 1, 'Lymphocytic interstitial pneumonia': 1, '

## FILTRA A TABELA PARA USO APENAS DAS CLASSES DE INTERESSE
Se general_case="remove":<br>
    - Se a classe da imagen não está em mapping, a imagem é removida da table<br>
Se general_case="subst":<br>
    - Se a classe da imagen não está em mapping, a classe da imagem é modificada para o valor em "std_subst"<br>
        Para este caso é possível passar uma lista ```remove_classes```, assim a classe não presente em mapping será substituída no caso padrão mas será removida da table se estiver presente na lista. <br>
        Ex.:<br>
        - class = "blabla" será atribuída o valor de mapping["std_subst"]<br>
        - class = "todo" terá a imagem removida da tabela<br>

In [6]:
mapping = {"COVID-19": "COVID-19",
           "COVID-19, ARDS": "COVID-19",
           "Normal": "Normal",
           "Pneumonia": "Pneumonia", # OBS.: Linda ignora essa classe
           "SARS": "Pneumonia",
           "MERS": "Pneumonia",
           "Streptococcus": "Pneumonia",
           "Klebsiella": "Pneumonia",
           "Chlamydophila": "Pneumonia",
           "Legionella": "Pneumonia",
           "Lung Opacity": "Pneumonia",
           "1": "Pneumonia",
           "std_subst": "Non-COVID"}
# remove_classes = ["todo", "nan", "Unknown"]

filtered_table = filter_table(file_table, mapping, general_case="remove")
table_info(filtered_table)

Total de imagens: 677
Contagem de cada classe por dataset: {'COVID-19': 607, 'Pneumonia': 70}


## SEPARAÇÃO DE TESTES PARA COHEN, FIG1, ACTUALMED E SIRM

In [7]:
test_patients = {"Pneumonia": ['8', '31'],
                 "COVID-19": ['19', '20', '36', '42', '86', 
                              '94', '97', '117', '132', 
                              '138', '144', '150', '163', '169', '174', '175', '179', '190', '191',
                              'COVID-00024', 'COVID-00025', 'COVID-00026', 'COVID-00027', 'COVID-00029',
                              'COVID-00030', 'COVID-00032', 'COVID-00033', 'COVID-00035', 'COVID-00036',
                              'COVID-00037', 'COVID-00038',
                              'ANON24', 'ANON45', 'ANON126', 'ANON106', 'ANON67',
                              'ANON153', 'ANON135', 'ANON44', 'ANON29', 'ANON201', 
                              'ANON191', 'ANON234', 'ANON110', 'ANON112', 'ANON73', 
                              'ANON220', 'ANON189', 'ANON30', 'ANON53', 'ANON46',
                              'ANON218', 'ANON240', 'ANON100', 'ANON237', 'ANON158',
                              'ANON174', 'ANON19', 'ANON195',
                              'COVID-19(119)', 'COVID-19(87)', 'COVID-19(70)', 'COVID-19(94)', 
                              'COVID-19(215)', 'COVID-19(77)', 'COVID-19(213)', 'COVID-19(81)', 
                              'COVID-19(216)', 'COVID-19(72)', 'COVID-19(106)', 'COVID-19(131)', 
                              'COVID-19(107)', 'COVID-19(116)', 'COVID-19(95)', 'COVID-19(214)', 
                              'COVID-19(129)']}

test_table = []
train_table = []
for row in filtered_table:
    if row["class"] in test_patients and row["id"] in test_patients[row["class"]]:
        test_table.append(row)
    else:
        train_table.append(row)
    
print(f"\nTRAIN:\n-------------------")
table_info(train_table)
print(f"\nTEST:\n-------------------")
table_info(test_table)


TRAIN:
-------------------
Total de imagens: 569
Contagem de cada classe por dataset: {'COVID-19': 504, 'Pneumonia': 65}

TEST:
-------------------
Total de imagens: 108
Contagem de cada classe por dataset: {'Pneumonia': 5, 'COVID-19': 103}


## FILTRAGEM DAS CLASSES DE INTERESSE DO RSNA

In [8]:
rsna_filtered_table = filter_table(rsna.table, mapping, general_case="remove")
table_info(rsna_filtered_table)

Total de imagens: 18406
Contagem de cada classe por dataset: {'Normal': 8851, 'Pneumonia': 9555}


## SEPARAÇÃO DE TESTES RSNA

In [9]:
split = 0.2
rsna_train, rsna_test = split_table(rsna_filtered_table, split)
print(f"\nTRAIN:\n-------------------")
table_info(rsna_train)
print(f"\nTEST:\n-------------------")
table_info(rsna_test)


TRAIN:
-------------------
Total de imagens: 14725
Contagem de cada classe por dataset: {'Pneumonia': 7674, 'Normal': 7051}

TEST:
-------------------
Total de imagens: 3681
Contagem de cada classe por dataset: {'Pneumonia': 1881, 'Normal': 1800}


## JUNTA AS TABLES DE TESTE E TREINO

In [10]:
train_table += rsna_train
test_table += rsna_test
#train_table = rsna_train
#test_table = rsna_test

print(f"\nTRAIN:\n-------------------")
table_info(train_table)
print(f"\nTEST:\n-------------------")
table_info(test_table)


TRAIN:
-------------------
Total de imagens: 15294
Contagem de cada classe por dataset: {'COVID-19': 504, 'Pneumonia': 7739, 'Normal': 7051}

TEST:
-------------------
Total de imagens: 3789
Contagem de cada classe por dataset: {'Pneumonia': 1886, 'COVID-19': 103, 'Normal': 1800}


## MONTA O DATASET COPIANDO OS ARQUIVOS DA TABLE (SEPARADOS POR CLASSE)

In [11]:
from ds_utils import mount_dataset

mounted_dataset_path = "./not_sync/target_dataset"
if not os.path.isdir(mounted_dataset_path):
    os.mkdir(mounted_dataset_path)
train_path = os.path.join(mounted_dataset_path, "train")
test_path = os.path.join(mounted_dataset_path, "test")

print(f"\nTRAIN:\n-------------------")
mount_dataset(train_path, train_table)
print(f"\nTEST:\n-------------------")
mount_dataset(test_path, test_table)


TRAIN:
-------------------
15294 imagens escritas no dataset.

TEST:
-------------------
3789 imagens escritas no dataset.


VERIFY COPIED FILES

In [12]:
verification_list = []
for root, dirs, files in os.walk(mounted_dataset_path):
    for f in files:
        verification_list.append(f)
print(len(verification_list))

16683


TRAINING FEATURES

In [62]:
image_size = (150, 150)
batch_size = 32
max_batches = 10
n_features = 1
color_mode = "grayscale"
class_mode = "categorical"


In [63]:
import tensorflow as tf

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

train_datagen = train_datagen.flow_from_directory(
    os.path.join(mounted_dataset_path, "train"),
    shuffle=True,
    target_size=image_size,
    color_mode=color_mode,
    batch_size=batch_size)
test_datagen = test_datagen.flow_from_directory(
    os.path.join(mounted_dataset_path, "test"),
    shuffle=True,
    target_size=image_size,
    color_mode=color_mode,
    batch_size=batch_size)



Found 13019 images belonging to 3 classes.
Found 3664 images belonging to 3 classes.


In [68]:
x_train = []
y_train = []

for i in range(max_batches):
    print(f"Batch: {i}")
    batch = train_datagen.next()
    x_train.extend(batch[0])
    y_train.extend(batch[1])
x_train = np.array(x_train)
y_train = np.array(y_train)


Batch: 0
Batch: 1
Batch: 2
Batch: 3
Batch: 4
Batch: 5
Batch: 6
Batch: 7
Batch: 8
Batch: 9


In [69]:
print(x_train.shape)
print(y_train.shape)

(320, 150, 150, 1)
(320, 3)


In [72]:
import autokeras as ak

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version='v2')(output_node)
output_node = ak.Merge()([output_node1, output_node2])
output_node = ak.ClassificationHead()(output_node)

model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    overwrite=True,
    max_trials=1)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    validation_split=0.2, 
    epochs=10,
    verbose=1)
print(model.evaluate(dataset))



Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_block_1/ke...|3                 |?                 
conv_block_1/nu...|2                 |?                 
conv_block_1/nu...|2                 |?                 
conv_block_1/se...|True              |?                 
conv_block_1/ma...|True              |?                 
conv_block_1/dr...|0                 |?                 
conv_block_1/fi...|32                |?                 
conv_block_1/fi...|32                |?                 
conv_block_1/fi...|32                |?                 
conv_block_1/fi...|32                |?                 
res_net_block_1...|False             |?                 
res_net_block_1...|resnet50_v2       |?                 
res_net_block_1...|False             |?                 
classification_...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 

Epo

KeyboardInterrupt: 